In [11]:
import torch
import torch.nn as nn
from torchvision import transforms
from pycocotools.coco import COCO
import numpy as np
from PIL import Image
import os
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel
from torch.nn import functional as F
import DETR_Diffusion
from torchmetrics.image.fid import FrechetInceptionDistance
from skimage.metrics import structural_similarity

In [12]:
class ModelEvaluator:
    def __init__(self, editor_model, original_prompt, batch_size=8):
        self.editor = editor_model
        self.batch_size = batch_size
        self.original_prompt = original_prompt
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # SSIM 모듈 초기화
        #self.ssim_module = pytorch_ssim.SSIM().to(self.device)
        
        # 이미지 전처리
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor()
        ])

    def load_coco_dataset(self, annotation_file, image_dir):
        """COCO 데이터셋 로드"""
        try:
            self.coco = COCO(annotation_file)
            self.image_dir = image_dir
            
            # 유효한 이미지 ID만 필터링
            self.img_ids = [img_id for img_id in self.coco.getImgIds() 
                          if self.coco.loadImgs(img_id) is not None]
            
            print(f"Total valid images: {len(self.img_ids)}")
            if len(self.img_ids) == 0:
                raise ValueError("No valid images found in the dataset")
                
        except Exception as e:
            print(f"Error loading COCO dataset: {e}")
            raise

    def get_image_and_prompt(self, img_id):
        """이미지와 편집 프롬프트 가져오기"""
        try:
            #print(img_id, type(img_id))
            imgs = self.coco.loadImgs(int(img_id))
            if not imgs:
                print(f"No image found for id: {img_id}")
                return None, None
                
            img_info = imgs[0]
            image_path = os.path.join(self.image_dir, img_info['file_name'])
            
            if not os.path.exists(image_path):
                print(f"Image file not found: {image_path}")
                return None, None
                
            image = Image.open(image_path).convert('RGB')
            
            # 이미지의 어노테이션 가져오기
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            anns = self.coco.loadAnns(ann_ids)
            
            if len(anns) > 0:
                cat_id = anns[0]['category_id']
                cat_name = self.coco.loadCats(cat_id)[0]['name']
                print(cat_id, cat_name)
                target_prompt = self.original_prompt.replace('target', cat_name)
                prompt = f"{cat_name} changes to {target_prompt}, high resolution"
            else:
                prompt = None
                
            return image, prompt
            
        except Exception as e:
            print(f"Error processing image {img_id}: {e}")
            return None, None

    def calculate_fid(self, real_images, edited_images, batch_size=8):
        """FID 점수 계산"""
        try:
            # FID 모듈 초기화
            fid = FrechetInceptionDistance(feature=2048, normalize=True).to(self.device)
            
            # 이미지들을 텐서로 변환하고 처리
            for real, edited in zip(real_images, edited_images):
                # PIL 이미지를 텐서로 변환
                real_tensor = self.transform(real).unsqueeze(0).to(self.device)
                edited_tensor = self.transform(edited).unsqueeze(0).to(self.device)
                
                # FID 업데이트
                fid.update(real_tensor, real=True)
                fid.update(edited_tensor, real=False)
            
            # FID 계산
            fid_value = fid.compute().item()
            
            return fid_value
            
        except Exception as e:
            print(f"FID 계산 중 에러 발생: {e}")
            return None

    def calculate_ssim(self, real_image, edited_image):
        """SSIM 점수 계산"""
        try:
            # PIL 이미지를 numpy 배열로 변환
            real_array = np.array(real_image)
            edited_array = np.array(edited_image)
            
            # 이미지 크기가 다르면 동일하게 맞추기
            if real_array.shape != edited_array.shape:
                edited_array = np.array(edited_image.resize(real_image.size))
            
            # SSIM 계산 (multichannel=True로 설정하여 RGB 이미지 처리)
            ssim_value = structural_similarity(
                real_array, 
                edited_array, 
                channel_axis=2,  # RGB 채널을 위한 축 지정
                data_range=255
            )
            
            return ssim_value
            
        except Exception as e:
            print(f"SSIM 계산 중 에러 발생: {e}")
            return None

    def evaluate(self, num_images=1000):
        """모델 평가 수행"""
        results = {
            'fid': None,
            'ssim_scores': [],
            'success_rate': 0,
            'processing_time': []
        }
        
        real_images = []
        edited_images = []
        successful_edits = 0
        
        # 이미지 샘플링 및 평가
        selected_ids = np.random.choice(self.img_ids, min(num_images, len(self.img_ids)), replace=False)
        
        for img_id in tqdm(selected_ids):
            image, prompt = self.get_image_and_prompt(img_id)
            
            if prompt is None:
                continue
                
            try:
                # 시간 측정 시작
                start_time = torch.cuda.Event(enable_timing=True)
                end_time = torch.cuda.Event(enable_timing=True)
                start_time.record()
                
                # 이미지 편집
                edited_image = self.editor.edit_image(image, prompt)
                
                # 시간 측정 종료
                end_time.record()
                torch.cuda.synchronize()
                processing_time = start_time.elapsed_time(end_time)
                
                if edited_image is not None:
                    real_images.append(image)
                    edited_images.append(edited_image)
                    successful_edits += 1
                    
                    # SSIM 계산
                    ssim_score = self.calculate_ssim(image, edited_image)
                    #print(ssim_score)
                    results['ssim_scores'].append(ssim_score)
                    results['processing_time'].append(processing_time)
                    
            except Exception as e:
                print(f"Error processing image {img_id}: {e}")
                
        # FID 계산
        if len(real_images) > 0:
            results['fid'] = self.calculate_fid(real_images, edited_images, self.batch_size)
            
        # 성공률 계산
        results['success_rate'] = successful_edits / len(selected_ids)
        
        # 결과 요약
        summary = {
            'fid_score': results['fid'],
            'avg_ssim': np.mean(results['ssim_scores']),
            'std_ssim': np.std(results['ssim_scores']),
            'success_rate': results['success_rate'],
            'avg_processing_time': np.mean(results['processing_time']),
            'total_images_processed': len(selected_ids)
        }
        
        return summary

In [13]:
def print_result(results):

    print("\nEvaluation Results:")
    print(f"FID Score: {results['fid_score']:.2f}")
    print(f"Average SSIM: {results['avg_ssim']:.4f} ± {results['std_ssim']:.4f}")
    print(f"Success Rate: {results['success_rate']*100:.2f}%")
    print(f"Average Processing Time: {results['avg_processing_time']:.2f}ms")
    print(f"Total Images Processed: {results['total_images_processed']}")

In [14]:
editor = DETR_Diffusion.ImageEditor()

Using cache found in /home/ubuntu/.cache/torch/hub/facebookresearch_detr_main
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PA

In [22]:
# COCO 데이터셋 설정
annotation_file = 'data/coco/annotations/instances_val2017.json'
image_dir = 'data/coco/images/val2017'

In [23]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="red target")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_red_100 = evaluator.evaluate(num_images=100)

loading annotations into memory...
Done (t=0.86s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/100 [00:00<?, ?it/s]

24 zebra
zebra 
 red zebra, high resolution


  1%|          | 1/100 [00:02<04:16,  2.59s/it]

65 bed
bed 
 red bed, high resolution


...

 98%|█████████▊| 98/100 [04:24<00:05,  2.72s/it]

51 bowl
bowl 
 red bowl, high resolution


 99%|█████████▉| 99/100 [04:27<00:02,  2.72s/it]

25 giraffe
giraffe 
 red giraffe, high resolution


100%|██████████| 100/100 [04:30<00:00,  2.70s/it]


In [24]:
print_result(results_red_100)


Evaluation Results:
FID Score: 105.73
Average SSIM: 0.6094 ± 0.1973
Success Rate: 100.00%
Average Processing Time: 2612.08ms
Total Images Processed: 100


In [25]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="red target")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_red_500 = evaluator.evaluate(num_images=500)

loading annotations into memory...
Done (t=1.39s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/500 [00:00<?, ?it/s]

1 person
person 
 red person, high resolution


  0%|          | 1/500 [00:02<22:33,  2.71s/it]

1 person
person 
 red person, high resolution


...

100%|█████████▉| 498/500 [22:31<00:05,  2.76s/it]

72 tv
tv 
 red tv, high resolution


100%|█████████▉| 499/500 [22:34<00:02,  2.75s/it]

7 train
train 
 red train, high resolution


100%|██████████| 500/500 [22:37<00:00,  2.71s/it]


In [26]:
print_result(results_red_500)


Evaluation Results:
FID Score: 53.09
Average SSIM: 0.6302 ± 0.1767
Success Rate: 99.20%
Average Processing Time: 2649.89ms
Total Images Processed: 500


In [27]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="target tilted slightly to the left")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_tilt_500 = evaluator.evaluate(num_images=500)

loading annotations into memory...
Done (t=0.45s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/500 [00:00<?, ?it/s]

1 person
person 
 person tilted slightly to the left, high resolution


  0%|          | 1/500 [00:02<23:25,  2.82s/it]

58 hot dog
hot dog 
 hot dog tilted slightly to the left, high resolution


...

100%|█████████▉| 498/500 [22:21<00:05,  2.66s/it]

1 person
person 
 person tilted slightly to the left, high resolution


100%|█████████▉| 499/500 [22:24<00:02,  2.68s/it]

77 cell phone
cell phone 
 cell phone tilted slightly to the left, high resolution


100%|██████████| 500/500 [22:27<00:00,  2.69s/it]


In [28]:
print_result(results_tilt_500)


Evaluation Results:
FID Score: 55.23
Average SSIM: 0.6193 ± 0.1759
Success Rate: 98.60%
Average Processing Time: 2646.67ms
Total Images Processed: 500


In [29]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="The target is facing the front")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_front_500 = evaluator.evaluate(num_images=500)

loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/500 [00:00<?, ?it/s]

19 horse
horse 
 The horse is facing the front, high resolution


  0%|          | 1/500 [00:03<25:12,  3.03s/it]

5 airplane
airplane 
 The airplane is facing the front, high resolution


...

100%|█████████▉| 498/500 [22:28<00:06,  3.23s/it]

88 teddy bear
teddy bear 
 The teddy bear is facing the front, high resolution


100%|█████████▉| 499/500 [22:31<00:03,  3.09s/it]

2 bicycle
bicycle 
 The bicycle is facing the front, high resolution


100%|██████████| 500/500 [22:33<00:00,  2.71s/it]


In [30]:
print_result(results_front_500)


Evaluation Results:
FID Score: 52.97
Average SSIM: 0.6305 ± 0.1718
Success Rate: 99.20%
Average Processing Time: 2643.55ms
Total Images Processed: 500


In [31]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="The target is upside down")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_updown_500 = evaluator.evaluate(num_images=500)

loading annotations into memory...
Done (t=0.43s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/500 [00:00<?, ?it/s]

44 bottle
bottle 
 The bottle is upside down, high resolution


  0%|          | 1/500 [00:03<25:21,  3.05s/it]

64 potted plant
potted plant 
 The potted plant is upside down, high resolution


...

100%|█████████▉| 498/500 [22:27<00:05,  2.74s/it]

28 umbrella
umbrella 
 The umbrella is upside down, high resolution


100%|█████████▉| 499/500 [22:30<00:02,  2.72s/it]

44 bottle
bottle 
 The bottle is upside down, high resolution


100%|██████████| 500/500 [22:34<00:00,  2.71s/it]


In [32]:
print_result(results_updown_500)


Evaluation Results:
FID Score: 54.21
Average SSIM: 0.6207 ± 0.1798
Success Rate: 99.00%
Average Processing Time: 2648.42ms
Total Images Processed: 500


In [33]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="red target")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_red_1000 = evaluator.evaluate(num_images=1000)

loading annotations into memory...
Done (t=0.45s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/1000 [00:00<?, ?it/s]

24 zebra
zebra 
 red zebra, high resolution


  0%|          | 1/1000 [00:02<44:04,  2.65s/it]

52 banana
banana 
 red banana, high resolution


...

100%|█████████▉| 998/1000 [45:00<00:05,  2.76s/it]

67 dining table
dining table 
 red dining table, high resolution


100%|█████████▉| 999/1000 [45:03<00:02,  2.60s/it]

23 bear
bear 
 red bear, high resolution


100%|██████████| 1000/1000 [45:05<00:00,  2.71s/it]


In [34]:
print_result(results_red_1000)


Evaluation Results:
FID Score: 36.23
Average SSIM: 0.6259 ± 0.1787
Success Rate: 99.00%
Average Processing Time: 2646.79ms
Total Images Processed: 1000


In [35]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="target tilted slightly to the left")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_tilt_1000 = evaluator.evaluate(num_images=1000)

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/1000 [00:00<?, ?it/s]

62 chair
chair 
 chair tilted slightly to the left, high resolution


  0%|          | 1/1000 [00:02<42:58,  2.58s/it]

9 boat
boat 
 boat tilted slightly to the left, high resolution


...

100%|█████████▉| 998/1000 [45:06<00:05,  2.72s/it]

77 cell phone
cell phone 
 cell phone tilted slightly to the left, high resolution


100%|█████████▉| 999/1000 [45:09<00:02,  2.69s/it]

72 tv
tv 
 tv tilted slightly to the left, high resolution


100%|██████████| 1000/1000 [45:12<00:00,  2.71s/it]


In [36]:
print_result(results_tilt_1000)


Evaluation Results:
FID Score: 34.13
Average SSIM: 0.6323 ± 0.1719
Success Rate: 98.90%
Average Processing Time: 2653.76ms
Total Images Processed: 1000


In [37]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="The target is facing the front")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_front_1000 = evaluator.evaluate(num_images=1000)

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/1000 [00:00<?, ?it/s]

1 person
person 
 The person is facing the front, high resolution


  0%|          | 1/1000 [00:02<46:08,  2.77s/it]

1 person
person 
 The person is facing the front, high resolution


...

100%|█████████▉| 998/1000 [45:11<00:05,  2.71s/it]

77 cell phone
cell phone 
 The cell phone is facing the front, high resolution


100%|█████████▉| 999/1000 [45:14<00:02,  2.72s/it]

4 motorcycle
motorcycle 
 The motorcycle is facing the front, high resolution


100%|██████████| 1000/1000 [45:17<00:00,  2.72s/it]


In [38]:
print_result(results_front_1000)


Evaluation Results:
FID Score: 33.89
Average SSIM: 0.6306 ± 0.1760
Success Rate: 99.00%
Average Processing Time: 2656.69ms
Total Images Processed: 1000


In [39]:
evaluator = ModelEvaluator(editor_model=editor, original_prompt="The target is upside down")
evaluator.load_coco_dataset(annotation_file, image_dir)
results_updown_1000 = evaluator.evaluate(num_images=1000)

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
Total valid images: 5000


  0%|          | 0/1000 [00:00<?, ?it/s]

44 bottle
bottle 
 The bottle is upside down, high resolution


  0%|          | 1/1000 [00:02<48:29,  2.91s/it]

64 potted plant
potted plant 
 The potted plant is upside down, high resolution


...

100%|█████████▉| 998/1000 [45:25<00:05,  2.72s/it]

1 person
person 
 The person is upside down, high resolution


100%|█████████▉| 999/1000 [45:27<00:02,  2.71s/it]

21 cow
cow 
 The cow is upside down, high resolution


100%|██████████| 1000/1000 [45:30<00:00,  2.73s/it]


In [40]:
print_result(results_updown_1000)


Evaluation Results:
FID Score: 37.02
Average SSIM: 0.6307 ± 0.1797
Success Rate: 99.40%
Average Processing Time: 2658.32ms
Total Images Processed: 1000
